In [23]:
# Modules
import subprocess
from IPython.display import clear_output
import pandas as pd
import re
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
import ast
from scipy.stats import norm
from scipy.optimize import minimize


In [24]:
# compile zdplaskin
def run_prep(inp_path):
    try:
        process = subprocess.Popen(
            'preprocessor.exe',
            stdout = subprocess.DEVNULL,        # ignore outputs
            stderr = subprocess.DEVNULL,        # ignore errors
            stdin = subprocess.PIPE,            # recognize input
            universal_newlines=True
        )
        
        process.stdin.write(inp_path)
        process.stdin.flush()                   # send a data

        while process.poll() is None:           # check the program state, if None, program is still in the run
            process.stdin.write('.\n')
            process.stdin.flush()
    except:
        pass
    print('check the run of preprocessor')
    return process

In [25]:
# Compile exe
def compile_zdp(name):
    compile_command = [
        'gfortran', '-o', name, 'dvode_f90_m.F90', 'zdplaskin_m.F90',
        'run_plasRxn_v2.F90', 'bolsig_x86_64_g.dll'
    ]
    
    try:
        subprocess.run(compile_command)
    except:
        pass
    print('check the compiler')

In [26]:
# Run exe
def run_exe(exe_path):
    try:
        process = subprocess.Popen(
            exe_path,
            stdout = subprocess.PIPE, # Read standard outputs
            stderr = subprocess.PIPE, # Read standard errors
            universal_newlines=True,  # outputs to str variables
            bufsize = 1               # control the size of buffer
        )

        log_flag = False             # The flag for starting log after "Caculation Start!!"
        while True:
            output = process.stdout.readline()
            if not output:
                break
            if "Calculation Start" in output:
                log_flag = True

            if log_flag:
                print(f'\r{output.strip()}           ',end='',flush=True)

            if "PRESS ENTER TO EXIT" in output:
                process.kill()        # forced shutdown
                break
            if "WARNING: BOLSIG+ convergence failed" in output:
                process.kill()        # forced shutdown
                break
    except:
        pass
    return process

In [27]:
# Error calculation
def cal_error(exp_result):
    # Read a result
    conditions = []
    with open('qt_conditions_list.txt','r') as file:
        for line in file:
            line = line.strip()
            line = line[2:]
            conditions.append(line)
        file.close()

    species = []
    with open('qt_species_list.txt','r') as file:
        for line in file:
            line = line.rstrip()
            line = line[3:]
            species.append(line)
        file.close()

    reactions = []
    reaction_list = pd.read_csv('parameter_set.csv')
    reactions = reaction_list['Reaction'].to_list()
    df_cd = pd.read_csv('qt_conditions.txt', sep=r'\s+', header=0, names=['Time [s]']+conditions)
    df_sp = pd.read_csv('qt_densities.txt', sep=r'\s+', header=0, names=['Time [s]']+species)
    df_rx = pd.read_csv('qt_rates.txt', sep=r'\s+', header=0, names=['Time [s]']+reactions)
    top_rate = df_rx.iloc[:,1:].sum().sort_values(ascending=False)

    CH4 = (df_sp['CH4'] + df_sp['CH4(V13)'] + df_sp['CH4(V24)'])
    C2H2 = (df_sp['C2H2'] + df_sp['C2H2(V13)']+ df_sp['C2H2(V2)']+ df_sp['C2H2(V5)'])
    C2H4 = (df_sp['C2H4'] + df_sp['C2H4(V1)']+ df_sp['C2H4(V2)'])
    C2H6 = (df_sp['C2H6'] + df_sp['C2H6(V13)']+ df_sp['C2H6(V24)'])
    C3H6 = (df_sp['C3H6'] + df_sp['C3H6(V)'])
    C3H8 = (df_sp['C3H8'] + df_sp['C3H8(V1)'] + df_sp['C3H8(V2)'])
    C4H10 = (df_sp['C4H9H'])
    C5H12 = (df_sp['C5H12'])
    H2 = df_sp['H2']
    C = df_sp['C']
    H = df_sp['H']
    CH = df_sp['CH']
    CH2 = df_sp['CH2']
    CH3 = df_sp['CH3']
    C2H3 = df_sp['C2H3']
    C2H5 = df_sp['C2H5']
    C3H5 = df_sp['C3H5']
    C3H7 = df_sp['C3H7']
    C4H9 = df_sp['C4H9']

    exp = exp_result
    
    sim_XCH4 = (CH4.iloc[0] - CH4.iloc[-1])/CH4.iloc[0]*100
    sim_SH2 = 0.5*H2.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H6 = 2*C2H6.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H4 = 2*C2H4.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H2 = 2*C2H2.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H8 = 3*C3H8.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H6 = 3*C3H6.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC4H10 = 4*C4H10.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC5H12 = 5*C5H12.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SH = 0.25*H.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SCH = CH.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SCH2 = CH2.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SCH3 = CH3.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H3 = 2*C2H3.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H5 = 2*C2H5.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H5 = 3*C3H5.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H7 = 3*C3H7.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC4H9 = 4*C4H9.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    # sim_SH2 = H2.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC2H6 = C2H6.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC2H4 = C2H4.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC2H2 = C2H2.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC3H8 = C3H8.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC3H6 = C3H6.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC4H10 = C4H10.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC5H12 = C5H12.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SH = H.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SCH = CH.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SCH2 = CH2.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SCH3 = CH3.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC2H3 = C2H3.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC2H5 = C2H5.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC3H5 = C3H5.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC3H7 = C3H7.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC4H9 = C4H9.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100


    sim = []
    sim.append(sim_XCH4)
    sim.append(sim_SH2)
    sim.append(sim_SC2H6)
    sim.append(sim_SC2H4)
    sim.append(sim_SC2H2)
    sim.append(sim_SC3H8)
    sim.append(sim_SC3H6)
    sim.append(sim_SC4H10)
    sim.append(sim_SC5H12)
    sim.append(sim_SH)
    sim.append(sim_SCH)
    sim.append(sim_SCH2)
    sim.append(sim_SCH3)
    sim.append(sim_SC2H3)
    sim.append(sim_SC2H5)
    sim.append(sim_SC3H5)
    sim.append(sim_SC3H7)
    sim.append(sim_SC4H9)

    sim2 = []
    sim2.append(sim_XCH4)
    sim2.append(sim_SH2)
    sim2.append(sim_SC2H6)
    sim2.append(sim_SC2H4)
    sim2.append(sim_SC2H2)
    sim2.append(sim_SC3H8)
    sim2.append(sim_SC3H6)
    sim2.append(sim_SC4H10)
    sim2.append(sim_SH)
    sim2.append(sim_SCH)
    sim2.append(sim_SCH2)
    sim2.append(sim_SCH3)
    sim2.append(sim_SC2H3)
    sim2.append(sim_SC2H5)
    sim2.append(sim_SC3H5)
    sim2.append(sim_SC3H7)
    sim2.append(sim_SC4H9)

    

    err =  float(np.sum((np.asarray(exp) - np.asarray(sim2))**2))
    
    return err, top_rate, sim, df_cd['Time [s]'].iloc[-1],df_sp.iloc[-1].sort_values(ascending=False)

    

In [28]:
result_list = ['XCH4','SH2','SC2H6','SC2H4','SC2H2','SC3H8','SC3H6','SC4','SC5+','SH','SCH','SCH2','SCH3','SC2H3','SC2H5','SC3H5','SC3H7','SC4H9']
exp_result = [18.873,74.645,14.318,1.739,1.556,3.968,0.555,0.508,0,0,0,0,0,0,0,0,0]
exp_result_expr = [18.873,74.645,14.318,1.739,1.556,3.968,0.555,0.508,0.711,0,0,0,0,0,0,0,0,0]

In [29]:
inp_path = f'kinet.inp\n'
exe_path = 'run_plasRxn_v2.exe'
prep_process = run_prep(inp_path)
prep_process.wait()
compile_zdp(exe_path)
exe_process = run_exe(exe_path)
exe_process.wait()
error, top_rate, sim_result, total_time, total_species = cal_error(exp_result)

check the run of preprocessor
check the compiler
PRESS ENTER TO EXIT ...           (s)           gured for BOLSIG+ solver exceeds 4.22D-01           

In [30]:
sim_result2 = [round(float(i),3) for i in sim_result]

In [31]:
df_res = pd.DataFrame({
    'res': result_list,
    'exp': exp_result_expr,
    'sim': sim_result2
    }
)
display(df_res)
print(total_time)
print(error)

,res,exp,sim
0,XCH4,18.873,25.248
1,SH2,74.645,78.049
2,SC2H6,14.318,8.688
3,SC2H4,1.739,1.121
4,SC2H2,1.556,1.772
5,SC3H8,3.968,4.183
6,SC3H6,0.555,0.406
7,SC4,0.508,4.286
8,SC5+,0.711,14.508
9,SH,0.000,0.009


3.5385
98.69603396220846


In [32]:
print(top_rate[:50])

H + H => H2                          3.841738e+21
e + CH => e + C + H                  3.207907e+21
e + CH4 => e + CH + H2 + H           3.194593e+21
e + CH4 => e + e + H + CH3^+         7.601802e+20
e + CH4 => e + CH3 + H               6.975856e+20
e + e + CH3^+ => e + CH3             6.485041e+20
CH3 + CH3 => C2H6                    6.275363e+20
e + C2H6 => e + C2H4 + H2            2.300612e+20
e + CH4 => e + CH4(v24)              1.873739e+20
C2H4 + H => C2H5                     1.799186e+20
e + CH4 => e + CH2 + H2              1.419506e+20
C2H5 + C2H5 => C4H9H                 1.181516e+20
CH4 + CH3^+ => H2 + C2H5^+           1.143798e+20
e + CH4 => e + CH4(v13)              1.130406e+20
e + e + C2H5^+ => e + C2H5           1.085659e+20
C4H9H + CH2 => C5H12                 9.890268e+19
CH3 + C2H5 => C3H8                   5.076308e+19
e + C2H4 => e + C2H2 + H2            2.221207e+19
e + CH4(v24) => e + CH + H2 + H      8.276186e+18
e + C2H5^+ => C2H3 + H + H           6.300226e+18


In [33]:
print(total_species[:50])

CH4          1.808400e+19
H2           9.710700e+18
C            4.041900e+18
C2H6         2.679300e+17
CH4(V24)     2.078100e+17
C5H12        1.805000e+17
CH4(V13)     1.262800e+17
C3H8         8.481600e+16
C4H9H        6.665000e+16
C2H2         5.494000e+16
C2H4         3.483000e+16
C3H6         8.339800e+15
CH3          2.845700e+15
H            2.310800e+15
C2H6(V24)    1.251000e+15
C2H6(V13)    1.056400e+15
C3H8(V1)     1.045900e+15
C3H8(V2)     8.824400e+14
C2H5         2.952800e+14
C2H2(V5)     1.225300e+14
C3H6(V)      8.831800e+13
C2H2(V2)     3.510700e+13
C2H2(V13)    2.466900e+13
C2H4(V1)     1.923200e+13
C2H3         1.872100e+13
C2H4(V2)     5.335200e+12
CH2          1.222500e+12
CH           5.819200e+09
E            5.918200e+05
CH3^+        5.036100e+05
C2H5^+       8.821200e+04
Time [s]     3.538500e+00
C3H7         6.387500e-02
C3H5         1.060900e-08
H^+          3.688700e-10
C4H9         1.419400e-11
C2H          1.485200e-14
CH2^+        1.243900e-15
CH4^+       